In [1]:
from xno.data.datasets.hdf5_dataset import H5pyDataset
from utils import MatReader
from torch.utils.data import DataLoader, TensorDataset, Dataset, default_collate

In [2]:
import torch
import matplotlib.pyplot as plt
import sys
from xno.models import XNO
from xno.data.datasets import load_darcy_flow_small
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalFNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss

In [3]:
data_path="/Users/sina/Documents/GitHub_Local/XNO/use_cases/data/1D_Lorenz_rho10.mat"

In [4]:
s = 2048
batch_size_train = 16
batch_size_vali = 16
batch_size_test = 16

In [5]:
reader = MatReader(data_path)
x_train = reader.read_field('f_train')
y_train = reader.read_field('u_train')
grid_x_train = reader.read_field('x_train')

x_vali = reader.read_field('f_vali')
y_vali = reader.read_field('u_vali')
grid_x_vali = reader.read_field('x_vali')

x_test = reader.read_field('f_test')
y_test = reader.read_field('u_test')
grid_x_test = reader.read_field('x_test') 

x_train = x_train.reshape(x_train.shape[0],s,1)
x_vali = x_vali.reshape(x_vali.shape[0],s,1)
x_test = x_test.reshape(x_test.shape[0],s,1)


In [6]:
# Define the custom Dataset
class DictDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {'x': self.x[idx], 'y': self.y[idx]}

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_vali.shape, y_vali.shape

In [9]:
x_train = x_train.permute(0, 2, 1)
y_train = y_train.unsqueeze(1)
x_vali = x_vali.permute(0, 2, 1)
y_vali = y_vali.unsqueeze(1)

In [10]:
train_loader = DictDataset(x_train, y_train)
test_loader = DictDataset(x_vali, y_vali)

In [11]:
train_loader = DataLoader(train_loader, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size_vali, shuffle=True)
test_loader = {
    2048: test_loader
}

In [ ]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

In [ ]:
batch = next(iter(test_loader[2048]))
type(test_loader), type(batch), batch['x'].shape, batch['y'].shape

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = XNO(
    max_n_modes=(16, ),
    n_modes=(2, ),
    hidden_channels=32,
    in_channels=1,
    out_channels=1,
    transformation="hno",
    # transformation_kwargs={"wavelet_level": 2, "wavelet_size": [2048]}
)
model = model.to(device)
n_params = count_model_params(model)

In [16]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [ ]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=2048,
    dataset_indices=[2],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

In [ ]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()

In [19]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [ ]:
trainer.train(
    train_loader,
    test_loader,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
)

In [ ]:
# FNO
{'train_err': 10.971330422621508,
 'avg_loss': 0.713136477470398,
 'avg_lasso_loss': None,
 'epoch_train_time': 4.013646583998707,
 '2048_h1': tensor(0.7238),
 '2048_l2': tensor(0.9066)}

In [ ]:
# HNO
{'train_err': 11.142363548278809,
 'avg_loss': 0.7242536306381225,
 'avg_lasso_loss': None,
 'epoch_train_time': 5.140133208000407,
 '2048_h1': tensor(0.7229),
 '2048_l2': tensor(0.7630)}

In [ ]:
# WNO
{'train_err': 6.489716823284443,
 'avg_loss': 0.42183159351348876,
 'avg_lasso_loss': None,
 'epoch_train_time': 5.868622500000129,
 '2048_h1': tensor(0.7528),
 '2048_l2': tensor(0.7417)}

In [ ]:
# LNO
{'train_err': 11.74565157523522,
 'avg_loss': 0.7634673523902893,
 'avg_lasso_loss': None,
 'epoch_train_time': 9.407964875001198,
 '2048_h1': tensor(0.7606),
 '2048_l2': tensor(0.7453)}